In [1]:
%%writefile Stock.py
  
# Streamlit app
def main():
    st.title('Stock Price Prediction App')
    ticker = st.text_input("Enter the company's ticker name:")
    
    if st.button('Predict Stock Prices'):
        start_date = pd.Timestamp.now() - pd.Timedelta(days=365)  # 1 year of historical data
        end_date = pd.Timestamp.now() - pd.Timedelta(days=1)  # Yesterday's date

        # Download stock data
        stock_data = download_stock_data(ticker, start_date, end_date)
        if stock_data is None:
            st.error(f"Failed to download data for {ticker}. Please enter a valid company ticker.")
            return

        # Preprocess data
        scaled_data, scaler = preprocess_data(stock_data)
        if scaled_data is None or scaler is None:
            st.error("Failed to preprocess data. Please try again.")
            return

        # Prepare training data
        x_train, y_train = prepare_training_data(scaled_data)
        if x_train is None or y_train is None:
            st.error("Failed to prepare training data. Please try again.")
            return

        # Train LSTM model
        st.subheader(f"{ticker} Stocks")
        model = train_lstm_model(x_train, y_train)
        if model is None:
            st.error("Failed to train the model. Please try again.")
            return

        # Predict stock price for the next 30 days
        predictions, dates = predict_stock_price(model, scaled_data, scaler)

        # Visualize stock prices
        st.subheader("Stock Prices and Predictions")
        plt.figure(figsize=(12, 6))
        plt.plot(stock_data.index, stock_data['Close'], label='Actual Price', color='blue')
        plt.plot(dates, predictions, label='Predicted Price', color='red', linestyle='--', marker='o')
        plt.title("Stock Prices and Predictions")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        plt.grid(True)
        st.pyplot(plt)

        # Print predicted closing prices for the next 30 days
        st.subheader("Predicted closing prices for the next 30 days:")
        for i, (price, date) in enumerate(zip(predictions, dates)):
            st.write(f"Day {i+1}: {price[0]} - {date.date()}")

if __name__ == "__main__":
    main()


Overwriting Stock.py
